In [316]:
import sys

sys.path.append('/Users/valuamba/projs/components_agent_sales/app/utils')
sys.path.append('/Users/valuamba/projs/components_agent_sales/app/core/clients/')

from html_messages_parser import get_element_messages, get_messages_from_html_file, select_json_block 
from famaga import FamagaClient

In [317]:
# let's define a few helper utilities

from pathlib import Path
import hashlib
import subprocess
import json
from bs4 import BeautifulSoup


text_cache = Path('cache')

def sha1(input_string):
    """Helper to hash input strings"""
    try:

        # Step 5: Create a new SHA-1 hash object
        hash_object = hashlib.sha1()

        # Step 6: Update the hash object with the bytes-like object
        hash_object.update(input_string.encode('utf-8'))

        # Step 7: Get the hexadecimal representation of the hash
        return hash_object.hexdigest()
    except Exception as e:
        raise ValueError(input_string) from e
        
def htmltotext(source: str):
    """Extract text from Email. Requires pdftotext binary in the path"""

    with open(source, 'r') as f:
        data = f.read()

    soup = BeautifulSoup(data, 'html.parser')
    
    return soup.get_text()

def extract_text(source):
    """Get text from Email. Cache results to avoid recomputation"""
    local = text_cache / sha1(source.name + " > text")

    if local.exists():
        return local
    print(f"extracting from {source}")
    print(f'target: {local}')
    htmltotext(source, local)
    return local


def email_to_text(path):
    return extract_text(path).read_text()

In [544]:
from functools import wraps
import inspect
import json
import pandas as pd

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, 'to_dict'):
            return obj.to_dict()
        if isinstance(obj, pd.Int64Dtype):
            return int(obj)  # Convert Int64 to a regular int
        return json.JSONEncoder.default(self, obj)

def stored(func):
    """
    implements nix-like durable memoisation of function results.

    Lazy way to avoid recomputing expensive calls. Expects results to be JSON-serializable
    """
    @wraps(func)
    def CACHE(*args, **kwargs):
        name = func.__name__
        meta = {}

        meta["name"] = name
        meta["func"] = inspect.getsource(func)
        meta["args"] = args
        meta["kwargs"] = kwargs

        js = json.dumps(meta, cls=CustomEncoder)  # Using CustomEncoder
        sha = hashlib.sha1(js.encode('utf-8'))

        digest = sha.hexdigest()

        path = text_cache / f"{digest}-{name}.json"

        if path.exists():
            with path.open('r') as r:
                cached = json.load(r)
            return cached["result"]
        result = func(*args, **kwargs)
        meta["result"] = result
        with path.open('w') as w:
            json.dump(meta, w, cls=CustomEncoder)  # Using CustomEncoder
        return result

    return CACHE

In [404]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
openai_client = OpenAI()

@stored
def get_gpt(content, model="gpt-4-1106-preview", temperature=0, max_tokens=1000, stream=True):
    """
    Cached call to GPT.
    """
    messages = [{"role": "user", "content": content}]

    if stream:
        response = openai_client.chat.completions.create(
            model=model, 
            messages=messages, 
            temperature=temperature, 
            stream=True
        )
        
        collected_messages = []
        for chunk in response:
            if chunk.choices[0].delta.content:
                print(chunk.choices[0].delta.content, end='')
                collected_messages.append(chunk.choices[0].delta.content)
    
        content_str = ''.join(collected_messages)
        return content_str
    else:
        completion = openai_client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
        )
        return completion.model_dump()


In [321]:
get_gpt('What is the lol kek cheburek?1', 'gpt-4')

'"Lol Kek Cheburek" is a phrase that originated from Russian internet slang. "Lol" is an English internet slang term meaning "laugh out loud", "Kek" is a Korean internet slang term used to express laughter, similar to "lol", and "Cheburek" is a popular deep-fried turnover with a filling of ground or minced meat and onions. It is often used in memes or as a humorous phrase online.'

In [ ]:
7f2eee719c076609a1f11f770bf9513f1c1f94e0-get_gpt

In [13]:
name1 = '502048_78717.0.html'
name = '467721_51407.0.html'

deal_id = name.split('_')[0]

text = htmltotext(Path(f'./deals_html/discount_v4/{name}'))

In [6]:
@stored
def get_messages(txt):

    prompt = """
You are Sales manager. Read a conversation between customer and manager and classify messages, senders (customer or manager).

Respond with array of messagds. Respond with an empty string, if none is found

```txt
$TXT
```""".strip() + "\n"
    
    filled = prompt.replace("$TXT", txt)
    result = get_gpt(filled)
    return result

In [35]:
result = get_messages(text)
result

'```json\n[\n  {\n    "message": "Please give the price in words for better understanding. also mention HSCode",\n    "sender": "customer"\n  },\n  {\n    "message": "We highly appreciate your offer, but prices are much higher than our budgeted figures. Hence request to offer special discount enabling us to get approval from our management. Delivery period should be shortened as much as possible.",\n    "sender": "customer"\n  },\n  {\n    "message": "Hello Mr Shumail, Supplier have offered for base please find attached the TDS and select the complete code. Can you please check the item number again? Unfortunately this is not complete.I have attached the Schaltbau catalog for you. You will find the order key on page 4. Thank you Best Regards, Pushkar Agarwal +97156.9576861",\n    "sender": "manager"\n  },\n  {\n    "message": "Offer-Nr.:502048 Customer request #:Shumail Customer #:95613 Date:Tuesday, 23 April 2024 Inquiry #:Contact person:Pushkar Agarwal Offer valid till22.05.2024 Inqu

In [38]:
@stored
def discount_chronology(txt):

    prompt = """
You are Sales manager. Read a conversation between customer and manager and classify discount chronology discussion.

Respond with array of messages where was mentioned or discussed discount.
Respond with an empty string, if none is found

```txt
$TXT
```""".strip() + "\n"
    
    filled = prompt.replace("$TXT", txt)
    result = get_gpt(filled)
    return result

In [44]:
messages = []
for msg in select_json_block(result):
    if 'Offer-Nr.' not in msg['message']:
        messages.append(msg)
    else:
        messages.append({ 'message': '< Commercial offer >', 'sender': 'maanger'})


chronology = discount_chronology('\n'.join([f'Message: {m['message']}\nSender: {m['sender']}' for m in messages]))     

```json
[
  "We highly appreciate your offer, but prices are much higher than our budgeted figures. Hence request to offer special discount enabling us to get approval from our management. Delivery period should be shortened as much as possible."
]
```

In [49]:
api_key = "YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw"
session_id = "085qpt4eflu39a0dg7hjhr5mdu"
famaga = FamagaClient(api_key, session_id)

def get_customer_data(deal_id):
    offer_info = famaga.list_offers_by_deal_id(deal_id)
    offer_id = offer_info['content'][0]['request']['id']
    client_id = offer_info['content'][0]['request']['firm']['id']
    
    purchase_history = famaga.get_client_purchase_history_formatted(client_id, int(deal_id))
    
    current_offer = famaga.list_current_offer_details(offer_id)

    return purchase_history, current_offer

In [50]:
get_customer_data(502048)

('[CLIENT PURCHASE HISTORY]\n**Client purchase history:**\nDate: 2022-09-29\n272381 (F862BP224K310Z Kemet/Evox Rifa) margin: 85%, sell: 22.47$ qty. 10, request id 350875\n\nDate: 2024-04-23\n375707 (T77630-40 AI-TEK Instruments) margin: 22%, sell: 3315.87$ qty. 1, request id 502049\n\nDate: 2024-04-25\n376251 (665006002 Mystik) margin: 25%, sell: 906.38$ qty. 1, request id 503139\n\n[/CLIENT PURCHASE HISTORY]',
 '(S800E Schaltbau) margin: 25%, sell: 101.59$ qty. 6')

In [ ]:
import requests
import uuid
import json

data = {
    "run_uuid": str(uuid.uuid4()),
    "deal_id": deal_id,
    "raw_text": text
}


response = requests.post('http://localhost:8013/v2/agent/discount/raw_text', data=json.dumps(data))

response.content

In [ ]:
print(json.dumps(response.json(), indent=2))

In [7]:
import requests

class FamagaClient:
    def __init__(self, api_token):
        self.api_token = api_token
        self.base_url = "https://api.famaga.org"

    def list_offers_by_deal_id(self, deal_id):
        url = f"{self.base_url}/requisition?request={deal_id}"
        headers = {'Authorization': self.api_token}
        response = requests.get(url, headers=headers)
        return response.json()

    def get_client_purchase_history_formatted(self, client_id, deal_id):
        url = f"{self.base_url}/requisition?client={client_id}"
        headers = {'Authorization': self.api_token}
        response = requests.get(url, headers=headers)
        return response.json()

    def list_current_offer_details(self, offer_id):
        url = f"{self.base_url}/requisition/{offer_id}/products"
        headers = {'Authorization': self.api_token}
        response = requests.get(url, headers=headers)
        return response.json()

# Helper function to calculate margin
def calculate_margin(total_purchase_price, total_selling_price):
    return ((total_selling_price - total_purchase_price) / total_purchase_price) * 100

# Decision-making function
def determine_discount(deal_id):
    client = FamagaClient("Bearer YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw")

    offer_info = client.list_offers_by_deal_id(deal_id)
    if not offer_info['content']:
        return "No offers found for this deal ID"

    offer_id = offer_info['content'][0]['request']['id']
    client_id = offer_info['content'][0]['request']['firm']['id']
    purchase_history = client.get_client_purchase_history_formatted(client_id, int(deal_id))
    current_offer = client.list_current_offer_details(offer_id)

    if not current_offer:
        return "No current offer details found"

    total_purchase_price = sum([float(product['price_buy_ru']) for product in current_offer])
    total_selling_price = sum([float(product['price_sell_ru']) for product in current_offer])
    current_margin = calculate_margin(total_purchase_price, total_selling_price)

    # Implement the discount logic
    has_purchased_current_product = any(deal['id'] == deal_id for deal in purchase_history['content'])
    if has_purchased_current_product:
        previous_offer = purchase_history['content'][0]
        previous_price = float(previous_offer['request']['cost'])
        if total_purchase_price <= previous_price:
            return f"Offer same price as last time: {previous_price}"
        else:
            new_price = total_purchase_price * 1.12
            return f"Set 12% margin price: {new_price}, explain manufacturer's price change"

    has_specified_desired_price = any(deal['target_price'] > 0 for deal in purchase_history['content'])
    if has_specified_desired_price:
        desired_price = next(deal['target_price'] for deal in purchase_history['content'] if deal['target_price'] > 0)
        if total_purchase_price <= desired_price:
            return f"Offer desired price: {desired_price}"
        else:
            new_price = total_purchase_price * 1.10
            return f"Set 10% margin price: {new_price}, explain maximum possible discount"

    if purchase_history['total'] > 0:
        if all(float(deal['request']['cost']) == total_purchase_price for deal in purchase_history['content']):
            return "Deny discount, margin did not change"
        else:
            previous_offers = [float(deal['request']['cost']) for deal in purchase_history['content']]
            if any(prev_price > total_purchase_price for prev_price in previous_offers):
                return "Ignore case, manufacturer's price increase while deciding"
            else:
                return "Give 2% discount due to previous discount"

    if purchase_history['total'] == 0:
        if purchase_history['content'][0]['request']['firm']['type'] == '1':
            return "Give 2% discount, ask for desired price"
        else:
            return "Deny discount, client is not a large company"

    if purchase_history['total'] > 10:
        return "Set 10% margin, candidate for blacklist if no purchase with max discount"

    if any(abs((deal['created'] - purchase_history['content'][0]['created']).days) <= 2 for deal in purchase_history['content']):
        return "Suggest discussing discount for multiple offers, pass to sales team for manual processing"

    return "No applicable discount found"

# # Example usage
# deal_id = 432036  # Replace with actual deal_id
# discount_decision = determine_discount(deal_id)
# print(discount_decision)


Ignore case, manufacturer's price increase while deciding


In [8]:
deals = [
    511087

]

discount_decision = determine_discount(deals[0])
print(discount_decision)

Ignore case, manufacturer's price increase while deciding


In [ ]:
def process_deals():
    client = FamagaClient("Bearer YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw")
    url = f"{client.base_url}/requisition?limit=200&page=20"
    headers = {'Authorization': client.api_token}
    response = requests.get(url, headers=headers)
    deals = response.json()['content']

    for deal in deals:
        deal_id = deal['request']['id']
        result = determine_discount(deal_id)
        print(f"Deal ID: {deal_id}, Discount Decision: {result}")

# Example usage
process_deals()

In [ ]:
Deal ID: 505143, Discount Decision: Give 2% discount due to previous discount
Deal ID: 505082, Discount Decision: Give 2% discount due to previous discount


In [13]:
import requests
import pandas as pd
from IPython.display import display, JSON

class FamagaClient:
    def __init__(self, api_token):
        self.api_token = api_token
        self.base_url = "https://api.famaga.org"

    def list_offers_by_deal_id(self, deal_id):
        url = f"{self.base_url}/requisition?request={deal_id}"
        headers = {'Authorization': self.api_token}
        response = requests.get(url, headers=headers)
        return response.json(), url

    def get_client_purchase_history_formatted(self, client_id, deal_id):
        url = f"{self.base_url}/requisition?client={client_id}"
        headers = {'Authorization': self.api_token}
        response = requests.get(url, headers=headers)
        return response.json(), url

    def list_current_offer_details(self, offer_id):
        url = f"{self.base_url}/requisition/{offer_id}/products"
        headers = {'Authorization': self.api_token}
        response = requests.get(url, headers=headers)
        return response.json(), url

def calculate_margin(total_purchase_price, total_selling_price):
    return ((total_selling_price - total_purchase_price) / total_purchase_price) * 100

def determine_discount(deal_id, client):
    trace_data = []
    
    offer_info, url = client.list_offers_by_deal_id(deal_id)
    trace_data.append({
        "API Call": "list_offers_by_deal_id",
        "URL": url,
        "Response": offer_info
    })

    if not offer_info['content']:
        return "No offers found for this deal ID", trace_data

    offer_id = offer_info['content'][0]['request']['id']
    client_id = offer_info['content'][0]['request']['firm']['id']
    
    purchase_history, url = client.get_client_purchase_history_formatted(client_id, int(deal_id))
    trace_data.append({
        "API Call": "get_client_purchase_history_formatted",
        "URL": url,
        "Response": purchase_history
    })
    
    current_offer, url = client.list_current_offer_details(offer_id)
    trace_data.append({
        "API Call": "list_current_offer_details",
        "URL": url,
        "Response": current_offer
    })

    if not current_offer:
        return "No current offer details found", trace_data

    total_purchase_price = sum([float(product['price_buy_ru']) for product in current_offer])
    total_selling_price = sum([float(product['price_sell_ru']) for product in current_offer])
    current_margin = calculate_margin(total_purchase_price, total_selling_price)

    has_purchased_current_product = any(deal['id'] == deal_id for deal in purchase_history['content'])
    if has_purchased_current_product:
        previous_offer = purchase_history['content'][0]
        previous_price = float(previous_offer['request']['cost'])
        if total_purchase_price <= previous_price:
            return f"Offer same price as last time: {previous_price}", trace_data
        else:
            new_price = total_purchase_price * 1.12
            return f"Set 12% margin price: {new_price}, explain manufacturer's price change", trace_data

    has_specified_desired_price = any(deal['target_price'] > 0 for deal in purchase_history['content'])
    if has_specified_desired_price:
        desired_price = next(deal['target_price'] for deal in purchase_history['content'] if deal['target_price'] > 0)
        if total_purchase_price <= desired_price:
            return f"Offer desired price: {desired_price}", trace_data
        else:
            new_price = total_purchase_price * 1.10
            return f"Set 10% margin price: {new_price}, explain maximum possible discount", trace_data

    if purchase_history['total'] > 0:
        if all(float(deal['request']['cost']) == total_purchase_price for deal in purchase_history['content']):
            return "Deny discount, margin did not change", trace_data
        else:
            previous_offers = [float(deal['request']['cost']) for deal in purchase_history['content']]
            if any(prev_price > total_purchase_price for prev_price in previous_offers):
                return "Ignore case, manufacturer's price increase while deciding", trace_data
            else:
                return "Give 2% discount due to previous discount", trace_data

    if purchase_history['total'] == 0:
        if purchase_history['content'][0]['request']['firm']['type'] == '1':
            return "Give 2% discount, ask for desired price", trace_data
        else:
            return "Deny discount, client is not a large company", trace_data

    if purchase_history['total'] > 10:
        return "Set 10% margin, candidate for blacklist if no purchase with max discount", trace_data

    if any(abs((deal['created'] - purchase_history['content'][0]['created']).days) <= 2 for deal in purchase_history['content']):
        return "Suggest discussing discount for multiple offers, pass to sales team for manual processing", trace_data

    return "No applicable discount found", trace_data

def process_deals():
    client = FamagaClient("Bearer YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw")
    url = f"{client.base_url}/requisition?limit=2"
    headers = {'Authorization': client.api_token}
    response = requests.get(url, headers=headers)
    deals = response.json()['content']

    all_trace_data = []
    results = []

    for deal in deals:
        deal_id = deal['request']['id']
        discount_decision, trace_data = determine_discount(deal_id, client)
        
        result = {
            "Deal ID": deal_id,
            "Client ID": deal['request']['firm']['id'],
            "Firm": deal['request']['firm']['title'],
            "Cost": deal['request']['cost'],
            "Discount Decision": discount_decision
        }
        results.append(result)
        all_trace_data.append(trace_data)

    # Display results in a pretty format
    df = pd.DataFrame(results)
    display(df)
    
    # Display trace data for each request
    for i, trace_data in enumerate(all_trace_data):
        print(f"Trace data for Deal ID {results[i]['Deal ID']}:")
        display(JSON(trace_data))

# Example usage
process_deals()


,Deal ID,Client ID,Firm,Cost,Discount Decision
0,511220,118684,Aptal DMCC,1626.34,"Ignore case, manufacturer's price increase whi..."
1,511087,127211,The Leicester Bearing Co.Ltd,404.49,"Ignore case, manufacturer's price increase whi..."


Trace data for Deal ID 511220:


<IPython.core.display.JSON object>

Trace data for Deal ID 511087:


<IPython.core.display.JSON object>

## V2

In [ ]:
def get_sales_history(self, client_id: int, deal_id: int) -> HistoryResponse:
        url = f"{BASE_URL}/calculation-history?request={deal_id}"
        response = requests.get(url, headers=self._get_headers())
        return self._handle_response(response)

In [472]:
from dataclasses import dataclass
from typing import List, Optional, Union, Any

@dataclass
class Firm:
    id: int
    title: str
    type: str
    conversion: str
    link: str


@dataclass
class Brand:
    id: int
    title: str
    conversion: str
    link: str

@dataclass
class Contractor:
    id: int
    title: str
    link: str
    created: str
    modified: str

@dataclass
class Curator:
    id: int
    name: str
    username: str
    email: str

@dataclass
class Request:
    id: int
    firm: Firm
    initials: str
    brand: Brand
    contractor: Contractor
    curator: Curator
    requestManagers: List[List]
    cost: str

@dataclass
class Proform:
    fam: Optional[int]

@dataclass
class HistoryContent:
    id: int
    request: Request
    created: str
    lastsend: str
    processing_time: str
    decision: int
    link: str
    target_price: int
    target_price_solved: bool
    state: bool
    proform: Proform

@dataclass
class HistoryResponse:
    content: List[HistoryContent]
    total: int
    limit: int
    page: int


## PRODUCTS 

@dataclass
class ProductDetail:
    id: int
    ri_id: int
    cc_id: int
    request_id: int
    brand_id: int
    articul: str
    articul_id: int
    contractor_id: int
    count: int
    price: str            # purchase price from manufacturer
    price_buy_de: float
    price_end: str        # the end price for client
    price_deliv: float 
    discount: str         # discount from supplier
    discount_client: float
    margin_ru: int        # margin that helps us identify discount
    tax: int
    price_buy_ru: float
    price_buy: float
    price_sell_ru: float
    price_sell: float
    unit: str
    weight: str
    planned_arrival_germany_from: int
    planned_arrival_germany_to: int
    confirmed_arrival_from: int
    confirmed_arrival_to: int
    checked_out: int
    checked_out_time: str
    created_by: int
    state: int
    ordering: int
    model_number: Optional[str] = ""
    description: Optional[str] = ""
    description_de: Optional[str] = ""
    brand_title: Optional[str] = None


## CALCULATION HISTORY

@dataclass
class Request:
    id: int

@dataclass
class User:
    id: int

@dataclass
class NoticeMailContent:
    id: int
    request: Request
    type: str
    user: User
    created_at: str
    note: str
    status: int

@dataclass
class NoticeMailResponse:
    content: List[NoticeMailContent]
    total: int
    limit: int
    offset: int

@dataclass
class Product:
    id: str
    tax: str
    count: str
    weight: str
    articul: str
    brand_id: str
    discount: str
    margin_ru: str
    description: str
    price_deliv: str
    model_number: str
    price_buy_de: str
    price_buy_ru: str
    price_sell_ru: str
    description_de: str
    expense: Optional[str] = None
    unit: Optional[str] = None

@dataclass
class VersionData:
    id: str
    bonus: str
    brand_id: str
    currency: str
    exchange: str
    products: List[Product]
    favourite: str
    firm_type: str
    margin_de: str
    freeze_end: int
    request_id: str
    bonus_sales: str
    brutto_open: str
    price_weigth: str
    total_weight: str
    contractor_id: str
    taxes_reserve: str
    term_validity: str
    bonus_purchase: str
    contractor_kun: str
    calculation_type: str
    delivery_cost_ru: str
    price_buy_all_de: str
    price_buy_all_ru: str
    price_sell_all_ru: str
    price_deliv_client: str
    contractor_kun_validity: str
    planned_arrival_germany_to: str
    price_buy_all_de_with_deliv: str
    planned_arrival_germany_from: str
    firm_manager: Optional[str] = None
    planned_arrival_unit: Optional[str] = None
    purchasing_department: List[str] = None
    calculation_error: Optional[str] = None
    request_type: Optional[str] = None

@dataclass
class CalculationHistoryContent:
    id: int
    request: Request
    user: User
    type: str
    created_at: str
    note: str
    sha1_hash: str
    version_data: Optional[VersionData] = None

@dataclass
class CalculationHistoryResponse:
    content: List[CalculationHistoryContent]
    total: int
    limit: int
    offset: int


from typing import Dict, List, Union, Optional
import logging
from dataclasses import dataclass
import requests
from dacite import from_dict


BASE_URL = "https://api.famaga.org"


class APIClient:
    def __init__(self, token: str) -> None:
        self.token = token

    def extract_date(self, datetime_str: str) -> str:
        dt = datetime.fromisoformat(datetime_str)
        return dt.strftime('%Y-%m-%d')

    def prettify_history_response(self, response: HistoryResponse) -> str:
        prettified = ""
        for content in response.content:
            prettified += (
                f"Deal: {content.request.id} at {self.extract_date(content.created)}\n"
            )
        return prettified

    def prettify_deal_offer(self, offer: HistoryContent):
        prettified = f"Deal: {offer.request.id} at {self.extract_date(offer.created)} | Decision: {offer.decision}"
        return prettified

    def prettify_offer_products(self, products: List[Product]) -> str:
        prettified = ""
        for product in products:
             prettified += (f'{product.id} ({product.articul}) {product.brand_id}, margin {product.margin_ru}%' +\
                            f' {round(float(product.price_sell_ru), 2)}\n')
        return prettified

    def prettify_product_details(self, products: List[ProductDetail]) -> str:
        prettified = ""
        for product in products:
             prettified += (f'{product.id} ({product.articul}) {product.brand_title}' +\
                            f' {round(float(product.price_end), 2)} ({product.discount}%)\n')
        return prettified

    def prettify_notice_mail_response(self, response: NoticeMailResponse) -> str:
        prettified = ""
        for content in response.content:
            prettified += (
                f"{content.request.id}"
                f"Type: {content.type}\n"
                f"User ID: {content.user.id}\n"
                f"Created At: {content.created_at}\n"
                f"Note: {content.note}\n"
                f"Status: {content.status}\n"
                "--------------------------\n"
            )
        return prettified
    
    def _get_headers(self) -> Dict[str, str]:
        return {"Authorization": f"Bearer {self.token}"}

    def _handle_response(self, response: requests.Response) -> HistoryResponse:
        try:
            response.raise_for_status()
            logging.info(f"Request to {response.url} succeeded with status code {response.status_code}")
            data = response.json()
            return from_dict(data_class=HistoryResponse, data=data)
        except requests.exceptions.HTTPError as e:
            logging.error(f"HTTP error occurred: {e}")
            raise
        except requests.exceptions.RequestException as e:
            logging.error(f"Error occurred: {e}")
            raise

    @stored
    def _offers_by_client_id(self, client_id) -> HistoryResponse:
        response = requests.get(f"{BASE_URL}/requisition?client={client_id}", headers=self._get_headers())
        return response.json()

    def offers_by_client_id(self, client_id) -> HistoryResponse:
        json_data = self._offers_by_client_id(client_id)
        return from_dict(data_class=HistoryResponse, data=json_data)

    @stored
    def _offers_by_deal_id(self, deal_id: int) -> HistoryResponse:
        url = f"{BASE_URL}/requisition?request={deal_id}"
        response = requests.get(url, headers=self._get_headers())
        return response.json()

    def offers_by_deal_id(self, deal_id: int) -> HistoryResponse:
        json_data = self._offers_by_deal_id(deal_id)
        return from_dict(data_class=HistoryResponse, data=json_data)

    @stored
    def _offer_products(self, offer_id: int) -> List[ProductDetail]:
        url = f"{BASE_URL}/requisition/{offer_id}/products"
        response = requests.get(url, headers=self._get_headers())
        return response.json()
    
    def offer_products(self, offer_id: int) -> List[ProductDetail]:
        json_data = self._offer_products(offer_id)
        return [ProductDetail(**product) for product in json_data]

    @stored
    def _get_notice_mail(self, request_id: int) -> NoticeMailResponse:
        url = f"{BASE_URL}/requisition-client-history?request={request_id}"
        response = requests.get(url, headers=self._get_headers())
        return response.json()

    def get_notice_mail(self, request_id: int) -> NoticeMailResponse:
        json_data = self._get_notice_mail(request_id)
        return from_dict(data_class=NoticeMailResponse, data=json_data)

    @stored
    def _get_calculation_history(self, request_id: int) -> CalculationHistoryResponse:
        url = f"{BASE_URL}/calculation-history?request={request_id}"
        response = requests.get(url, headers=self._get_headers())
        return response.json()

    def get_calculation_history(self, request_id: int) -> CalculationHistoryResponse:
        json_data = self._get_calculation_history(request_id)
        for c in json_data['content']:
            if c['version_data']:
                version_data = c['version_data']
                if isinstance(version_data['currency'], int):
                    version_data['currency'] = str(version_data['currency'])
                if isinstance(version_data['freeze_end'], str):
                    version_data['freeze_end'] = int(version_data['freeze_end'])
        return from_dict(data_class=CalculationHistoryResponse, data=json_data)

    def to_dict(self):
        return {
            "token": self.token
        }

In [473]:
client = APIClient('YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw')

In [ ]:
current_deal_id = 123

class DiscountStatement:
    hasSpecifiedDesiredPrice: bool
    desired_price: Optional[float] = None
    desired_discount: Optional[float] = None
    

class ConversationDiscountStatement
    deal_statement: DiscountStatement
    products_statement: Dict[str, DiscountStatement]

current_deal_offer = get_latest_offer(current_deal_id)
conversation_discount_statement = ConversationDiscountStatement()

new_deal_offer = copy.copy(current_deal_offer)

client_id = current_deal_offer.firm.id

purchase_history = get_customer_purchase_history(client_id)
current_deal_offer = get_latest_offer(current_deal_id)

bought_products_for_all_time = [deal.products for deal in purchase_history]

matched_products = get_matched_products(bought_products_for_all_time, current_deal_offer.products)

matched_products = {
    'AC1242': [
        {
            "purchase_price",
            "discount",
            "purchase_date"
        }
    ]
}



if matched_products.items():
    new_deal_offer = setup_previous_products_prices(new_deal_offer, matched_products)

    new_margin = calculate_deal_margin(new_deal_offer)
    if check_deal_margin(new_margin):
        pass
elif conversation_discount_statement.deal_statement.hasSpecifiedDesiredPrice:
    new_deal_offer = apply_deal_discount(new_deal_offer, conversation_discount_statement.desired_price)

    

In [486]:
def dataclass_to_df(data_class):
    if isinstance(data_class, list):
        return pd.DataFrame([asdict(item) for item in data_class])
    return pd.DataFrame([asdict(data_class)])

def history_response_to_df(history_response: HistoryResponse):
    # Convert the nested content list into a DataFrame
    content_df = dataclass_to_df(history_response.content)
    content_df = pd.json_normalize(content_df.to_dict(orient='records'))
    return content_df
    
    

offers = client.offers_by_client_id(client_id)

history_df = history_response_to_df(offers)


In [488]:
history_df.columns.tolist()

['id',
 'created',
 'lastsend',
 'processing_time',
 'decision',
 'link',
 'target_price',
 'target_price_solved',
 'state',
 'request.id',
 'request.firm.id',
 'request.firm.title',
 'request.firm.type',
 'request.firm.conversion',
 'request.firm.link',
 'request.initials',
 'request.brand.id',
 'request.brand.title',
 'request.brand.conversion',
 'request.brand.link',
 'request.contractor.id',
 'request.contractor.title',
 'request.contractor.link',
 'request.contractor.created',
 'request.contractor.modified',
 'request.curator.id',
 'request.curator.name',
 'request.curator.username',
 'request.curator.email',
 'request.requestManagers',
 'request.cost',
 'proform.fam']

In [ ]:

class CurrencyType:
    RUB = "RUB"

class BoughtProduct:
    purchase_price: float
    currency: CurrencyType
    

class ClientHistoryResolver:

    

In [450]:
calculation_history = client.get_calculation_history(413629)

In [447]:
client.offer_products(413629)

[ProductDetail(id=1095279, ri_id=783851, cc_id=497420, request_id=413629, brand_id=3044, articul='0000-2003-00', articul_id=819017, contractor_id=24715, count=1, price='645.53', price_buy_de=0, price_end='1043.56', price_deliv=0, discount='0', discount_client=0, margin_ru=12, tax=7, price_buy_ru=645.53, price_buy=0, price_sell_ru=1043.56, price_sell=0, unit='pcs', weight='3', planned_arrival_germany_from=0, planned_arrival_germany_to=0, confirmed_arrival_from=0, confirmed_arrival_to=0, checked_out=0, checked_out_time='0000-00-00 00:00:00', created_by=0, state=1, ordering=942850, model_number='', description='BOARD, PCB, SPROCKET SLIP 3PH DETECTOR', description_de='BOARD, PCB, SPROCKET SLIP 3PH DETECTOR', brand_title='Ross Hill'),
 ProductDetail(id=1111496, ri_id=0, cc_id=497420, request_id=413629, brand_id=0, articul='', articul_id=0, contractor_id=24715, count=1, price='42', price_buy_de=0, price_end='95', price_deliv=0, discount='0', discount_client=0, margin_ru=30, tax=0, price_buy_

In [ ]:
client_offers

In [ ]:
notice = client.get_notice_mail(413629)

print(json.dumps(asdict(notice), indent=2))

In [480]:
from dataclasses import dataclass, asdict
import json


client = APIClient('YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw')

deal_id = 413629
deal_id = 447706
deal_id = 447713
deal_id = 448583

YES_DECISIONS = [0, 1, 2]

def get_full_customer_history(deal_id: int):
    """Return customer data:
        - current deal offer
        - purchase deals history with products
        - calculation history with products
    """
    prettified = ''
    
    deal_info = client.offers_by_deal_id(deal_id)
    latest_offer = get_latest_offer(deal_id)
    
    prettified += f'## Latest deal({deal_id}) offer\n'
    prettified += client.prettify_offer_products(latest_offer.products)
    prettified += '\n'
    
    client_id = deal_info.content[0].request.firm.id
    
    client_offers = client.offers_by_client_id(client_id)

    paid_offers = [offer for offer in client_offers.content if offer.decision in YES_DECISIONS]
    
    prettified += '## Purchase history\n'
    for offer in paid_offers:
        if offer.request.id == 381927:
            continue
        prettified += client.prettify_deal_offer(offer)
        prettified += '\n'
        offer_products = client.offer_products(offer.request.id)
        prettified += client.prettify_product_details(offer_products)
        prettified += '\n'
    
        deal_id = offer.request.id
        
        calculation_history = client.get_calculation_history(deal_id)
        notice = client.get_notice_mail(deal_id)
        
        calc_hist = filter_calculations_by_notice_time(calculation_history.content, notice.content)
        
        for calculation in calc_hist:
            if calculation.version_data:
                prettified += f'   Calculation {calculation.id} at {calculation.created_at}\n'    
                prettified += f'      {client.prettify_offer_products(calculation.version_data.products)}\n'

    return prettified


prettified = get_full_customer_history(deal_id)

In [481]:
print(prettified)

## Latest deal(448583) offer
1162037 (HPQ330328S) 17821, margin 20% 572.31

## Purchase history
Deal: 448583 at 2023-10-24 | Decision: 2
1162037 (HPQ330328S) Hy-Pro 572.31 (15%)

   Calculation 2717648 at 2023-10-24T22:47:52+00:00
      1162037 (HPQ330328S) 17821, margin 20% 572.31

Deal: 421312 at 2023-07-11 | Decision: 2
1135215 (0820215114) Aventics (brand of Emerson) 410.99 (0%)

   Calculation 2639206 at 2023-08-28T12:50:52+00:00
      1104384 (0820215114) 8182, margin 18% 407.08

   Calculation 2564150 at 2023-07-11T15:25:50+00:00
      1104384 (0820215114) 8182, margin 20% 416.88




In [469]:

for calculation in calculation_history.content:
    if calculation.version_data:
        products = calculation.version_data.products
        print(client.prettify_offer_products(products))

1041991 (KL120) 4278, margin 20% 123.75

1041991 (KL120) 4278, margin 20% 83.75

1041991 (KL120) 4278, margin 20% 83.75

1041991 (KL120) 4278, margin 20% 83.75

1041991 (KL120                         ) 4278, margin 20% 83.75

1041991 (9317903) 4278, margin 0.00% 0.0

1041991 (9317903) 4278, margin 0.00% 0.0
1041992 (9099809) 4278, margin 0.00% 0.0



In [ ]:
calculation_history

In [ ]:
deal_info = client.offers_by_deal_id(413629)

client_id = deal_info.content[0].request.firm.id

In [258]:
client_offers = client.offers_by_client_id(client_id)

In [282]:
client_offers.content[0].request.id

467995

In [283]:
client.prettify_deal_offer(client_offers.content[0])

'Deal: 374953 at 2023-01-06'

In [ ]:
print('## Purchase history')
for offer in client_offers.content:
    print(client.prettify_deal_offer(offer))
    offer_products = client.offer_products(offer.request.id)
    print(client.prettify_product_details(offer_products))
    print()

In [295]:
latest_offer = get_latest_offer(413629)

print(f'Latest deal({413629}) offer')
print(client.prettify_offer_products(latest_offer.products))

Latest deal(413629) offer
1095279 (0000-2003-00) BOARD, PCB, SPROCKET SLIP 3PH DETECTOR, margin 14% 1066.43



In [ ]:
latest_offer

In [279]:
client.prettify_deal_offer(client_offers.content[0])

'Deal: 374953 at 2023-01-06'

In [250]:
print(client.prettify_history_response(deal_info))
print(client.prettify_product_details(client.offer_products(413629)))

Deal: 413629 at 2023-06-26T13:23:08+00:00

1095279 (0000-2003-00) Ross Hill 1043.56 (0%)
1111496 () None 95.0 (0%)



In [303]:
products_bought_before = get_products_bought_before(413629)

In [304]:
print('## Poducts bought brefore')

print(prettify_product_history(products_bought_before))

## Poducts bought brefore
1000367 (692.913007141) 738, margin 22% 419.17
1008100 () 0, margin 22% 127.0



In [482]:
res = get_gpt(f"""
Here is an instruction:

1. **Customer previously purchased THIS product:** Try to offer the same price as last time if profitability allows. If not, set a 12% margin and explain that the supplier's price has changed, and this is the maximum possible discount.

2. **Customer has NOT previously purchased THIS product but specified a desired price:** Offer a discount if the margin allows. Otherwise, set a 10% margin and explain that this is the maximum possible discount.

3. **Customer has NOT purchased THIS product and has NOT specified a desired price, BUT has purchased other products:**
   
   a) **If the margin was unchanged in previous deals** (the price in the initial offer equals the price in the final offer for the same quantity within one quotation): Refuse the discount.

   b) **If the margin changed in previous deals** (the price in the initial offer does not equal the price in the final offer for the same quantity within one quotation):
   
      - If the price in the final offer is higher: Ignore the case (this is due to the supplier's price increase while the customer was deciding).
      - If a discount was given previously: Offer a 2% discount.

4. **Customer has NEVER purchased before and has NOT specified a desired price:** Check the total number of customer deals.
   
   a) **First quotation for a large company:** Offer a 2% discount and ask for the desired price.
   
   b) **First quotation for a non-large company:** Refuse the discount.
   
   c) **More than 10 quotations without any sales:** Set a 10% margin. This customer is a candidate for the blacklist; if they do not buy even with the maximum discount, we will likely not work with them in the future.
   
   d) **Several quotations with request dates differing by +/- 2 days (likely one project):** Propose a discount if the customer buys multiple quotations at once. Do not specify the exact price but ask for the desired price and hand it over to the manager for manual processing.

Additionally, remember the outcomes:
   
   a) **In case of a FULL discount refusal:** If the customer still buys, continue to refuse discounts. If the customer does not buy, offer a 2% discount next time and ask for the desired price.
   
   b) **In case of offering a 2% discount:** If the customer buys, continue offering 2% where possible. If the customer does not buy, offer a 3% discount next time (if margin allows) and continue to find the "acceptable discount" from deal to deal.

# Customer data

Is specified desired price: False
Is customer large company: False

{prettified}

Please make decision

""", 'gpt-4o'
)

Based on the provided instruction and customer data, here is the decision-making process:

1. **Customer has NOT purchased THIS product and has NOT specified a desired price, BUT has purchased other products:**

   - The customer has purchased other products (Deal: 421312 and Deal: 448583).
   - We need to check if the margin was unchanged in previous deals.

2. **Checking previous deals:**

   - **Deal: 448583**:
     - Initial offer: 1162037 (HPQ330328S) 17821, margin 20% 572.31
     - Final offer: 1162037 (HPQ330328S) Hy-Pro 572.31 (15%)
     - The margin changed from 20% to 15%, and a discount was given.

   - **Deal: 421312**:
     - Initial offer: 1104384 (0820215114) 8182, margin 20% 416.88
     - Final offer: 1135215 (0820215114) Aventics (brand of Emerson) 410.99 (0%)
     - The margin changed from 20% to 0%, and a discount was given.

3. **Decision based on previous deals:**

   - Since the margin changed in previous deals and discounts were given, we should offer a 2% discou

In [421]:
print(res)

Based on the provided instructions and customer data, here is the decision-making process:

1. **Customer has NOT purchased THIS product and has NOT specified a desired price, BUT has purchased other products:**

   - **If the margin was unchanged in previous deals** (the price in the initial offer equals the price in the final offer for the same quantity within one quotation): Refuse the discount.
   - **If the margin changed in previous deals** (the price in the initial offer does not equal the price in the final offer for the same quantity within one quotation):
     - If the price in the final offer is higher: Ignore the case (this is due to the supplier's price increase while the customer was deciding).
     - If a discount was given previously: Offer a 2% discount.

### Analysis of Purchase History:

- **Deal 467995 (2024-01-05):** 
  - Product: 1197848 (UM200)
  - Initial offer: 1558.75 (20% margin)
  - Final offer: 1689.48 (23% margin)
  - **Margin changed** (final offer is hig

In [328]:
calcilation_response = client.get_calculation_history(413629)
notice_resp = client.get_notice_mail(413629)

get_products_with_changed_price(calcilation_response.content, notice_resp.content)

1043.56

In [ ]:
calcilation_response

In [203]:

print(client.prettify_history_response(deal_info))

ID: 318963
Request ID: 413629
Created: 2023-06-26T13:23:08+00:00
Last Sent: 2023-06-26T13:42:21+00:00
Processing Time: 2397957
Decision: 2
Link: index.php?option=com_oscrm&view=requisition&request_id=413629&Itemid=219
Target Price: 0
Target Price Solved: False
State: True
Proform: 53711
--------------------------



In [ ]:
def calculate_common_deal_margin(products: List[Product]) -> float:
    total_margin = sum(float(product.margin_ru) for product in products)
    return total_margin

def calculate_common_deal_margin_v2(products: List[Product]) -> float:
    total_purchase_price = sum(float(product.price_buy_ru) * float(product.count) for product in products)
    total_selling_price = sum(float(product.price_sell_ru) * float(product.count) for product in products)
    
    # Calculate margin using the provided formula
    margin = 100 * (1 - (total_purchase_price / total_selling_price))
    
    return margin


for content in client.offers_by_client_id(client_id).content:

    print(f'Brand title: {content.request.brand.title}. Requisition {content.request.id}')
    products = client.offer_products(content.request.id)

    print(f'Margin {calculate_common_deal_margin(products)}')
    print(f'Margin V2 {calculate_common_deal_margin_v2(products)}')
    for product in products:
        print(f'{product.articul} {product.brand_title}')
    # for branch in content.request

In [ ]:
calcilation_response = client.get_calculation_history(413629)

In [147]:
notice_resp = client.get_notice_mail(413629)

In [338]:
deal_id = client_offers.content[3].request.id
calcilation_response = client.get_calculation_history(deal_id)
notice_resp = client.get_notice_mail(deal_id)

get_products_with_changed_price(calcilation_response.content, notice_resp.content)

{'1121584': <PriceState.PRICE_NOT_CHANGED: 'PriceNotChanged'>}

In [148]:
from datetime import datetime, timedelta
from typing import List

def filter_calculations_by_notice_time(calculations: List[CalculationHistoryContent], notices: List[NoticeMailContent], time_delta_minutes: int = 5) -> List[CalculationHistoryContent]:
    filtered_calculations = []
    time_delta = timedelta(minutes=time_delta_minutes)

    for notice in notices:
        notice_time = datetime.fromisoformat(notice.created_at.replace('Z', '+00:00'))
        for calculation in calculations:
            calculation_time = datetime.fromisoformat(calculation.created_at.replace('Z', '+00:00'))
            if abs((calculation_time - notice_time).total_seconds()) <= time_delta.total_seconds():
                filtered_calculations.append(calculation)

    return filtered_calculations

In [ ]:
filter_calculations_by_notice_time(calcilation_response.content, notice_resp.content)

In [151]:
def compare_product_prices(first_offer: VersionData, latest_offer: VersionData):
    product_prices_comparison = {}

    first_offer_products = {product.id: float(product.price_sell_ru) for product in first_offer.products}
    latest_offer_products = {product.id: float(product.price_sell_ru) for product in latest_offer.products}

    for product_id, first_price in first_offer_products.items():
        latest_price = latest_offer_products.get(product_id)
        if latest_price is not None:
            if latest_price < first_price:
                comparison = "less"
            elif latest_price > first_price:
                comparison = "greater"
            else:
                comparison = "same"
            product_prices_comparison[product_id] = {
                "first_price": first_price,
                "latest_price": latest_price,
                "comparison": comparison
            }

    return product_prices_comparison

In [155]:
filtered_calculations = filter_calculations_by_notice_time(calcilation_response.content, notice_resp.content)

def get_products_with_changed_price(calculations: List[CalculationHistoryContent]):
    sorted_calculations = sorted(filtered_calculations, key=lambda x: x.created_at)
    if sorted_calculations:
        first_offer = sorted_calculations[0].version_data
        latest_offer = sorted_calculations[-1].version_data
    
        if first_offer and latest_offer:
            comparison_result = compare_product_prices(first_offer, latest_offer)
            for product_id, comparison in comparison_result.items():
                print(f"Product ID: {product_id}, First Price: {comparison['first_price']}, Latest Price: {comparison['latest_price']}, Comparison: {comparison['comparison']}")

get_products_with_changed_price(filtered_calculations)

Product ID: 1095279, First Price: 1141.91, Latest Price: 1066.43, Comparison: less


In [157]:
from datetime import datetime, timedelta
from typing import List, Dict
from enum import Enum

class PriceState(Enum):
    PRICE_NOT_CHANGED = "PriceNotChanged"
    PRICE_INCREASED = "PriceIncreased"
    PRICE_DECREASED = "PriceDecreased"

def filter_calculations_by_notice_time(calculations: List[CalculationHistoryContent], notices: List[NoticeMailContent], time_delta_minutes: int = 5) -> List[CalculationHistoryContent]:
    filtered_calculations = []
    time_delta = timedelta(minutes=time_delta_minutes)

    for notice in notices:
        notice_time = datetime.fromisoformat(notice.created_at.replace('Z', '+00:00'))
        for calculation in calculations:
            calculation_time = datetime.fromisoformat(calculation.created_at.replace('Z', '+00:00'))
            if abs((calculation_time - notice_time).total_seconds()) <= time_delta.total_seconds():
                filtered_calculations.append(calculation)

    return filtered_calculations

def compare_product_prices(first_offer: VersionData, latest_offer: VersionData) -> Dict[str, PriceState]:
    product_prices_comparison = {}

    first_offer_products = {product.id: float(product.price_sell_ru) for product in first_offer.products}
    latest_offer_products = {product.id: float(product.price_sell_ru) for product in latest_offer.products}

    for product_id, first_price in first_offer_products.items():
        latest_price = latest_offer_products.get(product_id)
        if latest_price is not None:
            if latest_price < first_price:
                state = PriceState.PRICE_DECREASED
            elif latest_price > first_price:
                state = PriceState.PRICE_INCREASED
            else:
                state = PriceState.PRICE_NOT_CHANGED
            product_prices_comparison[product_id] = state

    return product_prices_comparison

def get_products_with_changed_price(calculations: List[CalculationHistoryContent], notices: List[NoticeMailContent]) -> Dict[str, PriceState]:
    "Get "
    filtered_calculations = filter_calculations_by_notice_time(calculations, notices)
    sorted_calculations = sorted(filtered_calculations, key=lambda x: x.created_at)
    
    if sorted_calculations:
        first_offer = sorted_calculations[0].version_data
        latest_offer = sorted_calculations[-1].version_data
    
        if first_offer and latest_offer:
            comparison_result = compare_product_prices(first_offer, latest_offer)
            return comparison_result
    
    return {}

In [176]:
def get_latest_offer(request_id: int) -> Optional[VersionData]:
    "Get latest offer that was sent client on the current deal"
    calcilation_response = client.get_calculation_history(request_id)
    notice_resp = client.get_notice_mail(request_id)
    
    filtered_calculations = filter_calculations_by_notice_time(calcilation_response.content, notice_resp.content)
    sorted_calculations = sorted(filtered_calculations, key=lambda x: x.created_at)

    latest_offer = sorted_calculations[-1].version_data
    return latest_offer

def has_bought_product_before(client_id: int, product_id: int) -> bool:
    """Check if the customer has already bought the same product earlier."""
    history_response = self.offers_by_client_id(client_id)
    for content in history_response.content:
        products = self.offer_products(content.request.id)
        if any(product.id == product_id for product in products):
            return True
    return False

In [190]:
latest_offer = get_latest_offer(413629)

In [313]:
def get_full_purchase_history(client_id: int) -> List[ProductDetail]:
    """Retrieve the full purchase history for a client."""
    history_response = client.offers_by_client_id(client_id)
    all_products = []
    for content in history_response.content:
        all_products.extend(client.offer_products(content.request.id))
    return all_products
    

@dataclass
class ProductHistory:
    id: str
    tax: str
    count: str
    weight: str
    articul: str
    brand_id: str
    discount: str
    margin_ru: str
    description: str
    price_deliv: str
    model_number: str
    price_buy_de: str
    price_buy_ru: str
    price_sell_ru: str
    description_de: str
    previous_price: Optional[str] = None
    previous_margin: Optional[str] = None
    expense: Optional[str] = None
    unit: Optional[str] = None

def prettify_product_history(product_history: List[ProductHistory]):
    prettified = ""
    for product in products:
        prettified += (f'{product.id} ({product.articul}) {product.brand_id}, margin {product.margin_ru}%' +\
                            f' {round(float(product.price_sell_ru), 2)}\n')
    return prettified

def get_products_bought_before(deal_id: int) -> List[ProductHistory]:
    """Check which products from the latest offer were bought before and return their previous price and margin."""
    deal_info = client.offers_by_deal_id(deal_id)

    client_id = deal_info.content[0].request.firm.id
    if client_id is None:
        return []

    latest_offer = get_latest_offer(deal_id)
    if latest_offer is None:
        return []

    # Get full purchase history for the client
    purchase_history = get_full_purchase_history(client_id)
    purchased_product_ids = {product.articul: product for product in purchase_history}
    print(purchased_product_ids)

    # Check which products in the latest offer were bought before
    bought_before = []
    for product in latest_offer.products:
        if product.articul in purchased_product_ids:
            previous_product = purchased_product_ids[product.articul]
            product_history = ProductHistory(
                id=product.id,
                tax=product.tax,
                count=product.count,
                weight=product.weight,
                articul=product.articul,
                brand_id=product.brand_id,
                discount=product.discount,
                margin_ru=product.margin_ru,
                description=product.description,
                price_deliv=product.price_deliv,
                model_number=product.model_number,
                price_buy_de=product.price_buy_de,
                price_buy_ru=product.price_buy_ru,
                price_sell_ru=product.price_sell_ru,
                description_de=product.description_de,
                previous_price=previous_product.price_sell_ru,
                previous_margin=previous_product.margin_ru,
                expense=product.expense,
                unit=product.unit
            )
            bought_before.append(product_history)

    return bought_before

In [ ]:
print(prettify_product_history(get_products_bought_before(413629)))

In [ ]:
def calculate_common_margin(deal: VersionData) -> float:
    """Calculate the common margin of the deal."""
    total_cost = sum(float(product.price_buy_ru) for product in deal.products)
    total_revenue = sum(float(product.price_sell_ru) for product in deal.products)
    common_margin = (total_revenue - total_cost) / total_revenue * 100
    return common_margin

def check_margin_with_previous_prices(deal_id: int) -> bool:
    """Check if setting the previous price and margin for bought before products affects the overall margin."""
    latest_offer = get_latest_offer(deal_id)
    if latest_offer is None:
        return False

    products_bought_before = get_products_bought_before(deal_id)
    
    # Calculate total cost and revenue with previous prices and margins
    total_cost_with_prev = 0
    total_revenue_with_prev = 0
    
    for product in latest_offer.products:
        if any(prev_product.id == product.id for prev_product in products_bought_before):
            previous_product = next(prev_product for prev_product in products_bought_before if prev_product.id == product.id)
            total_cost_with_prev += float(previous_product.price_buy_ru)
            total_revenue_with_prev += float(previous_product.previous_price)
        else:
            total_cost_with_prev += float(product.price_buy_ru)
            total_revenue_with_prev += float(product.price_sell_ru)
    
    common_margin_with_prev = (total_revenue_with_prev - total_cost_with_prev) / total_revenue_with_prev * 100
    print(common_margin_with_prev)
    
    return common_margin_with_prev > 10

In [ ]:
def apply_discounts_and_check_margin(deal: VersionData, desired_discount) -> Union[VersionData, str]:
    """Apply discounts and check if the overall margin is greater than 10%."""
    latest_offer = get_latest_offer(deal.id)
    if latest_offer is None:
        return "No latest offer found."

    products_bought_before = get_products_bought_before(deal.id)
    
    # Apply previous prices and margins for products bought before
    for product in latest_offer.products:
        matching_previous_product = next((prev_product for prev_product in products_bought_before if prev_product.id == product.id), None)
        if matching_previous_product:
            product.price_sell_ru = matching_previous_product.previous_price
            product.margin_ru = matching_previous_product.previous_margin
        elif product.desired_price:
            # Apply desired price if product was not bought before
            desired_price = float(product.desired_price)
            cost_price = float(product.price_buy_ru)
            margin = ((desired_price - cost_price) / desired_price) * 100
            if margin >= 10:
                product.price_sell_ru = str(desired_price)
                product.margin_ru = str(margin)
            else:
                # Set margin to 10% if desired margin is not achievable
                product.margin_ru = "10"
                product.price_sell_ru = str(cost_price * 1.10)
    
    # Recalculate the common margin
    total_cost_with_discounts = sum(float(product.price_buy_ru) for product in latest_offer.products)
    total_revenue_with_discounts = sum(float(product.price_sell_ru) for product in latest_offer.products)
    common_margin_with_discounts = (total_revenue_with_discounts - total_cost_with_discounts) / total_revenue_with_discounts * 100
    
    # Check if the common margin meets the desired discount for the entire deal
    if deal.desired_discount and common_margin_with_discounts >= deal.desired_discount:
        return latest_offer
    else:
        return f"Margin is below the desired discount: {common_margin_with_discounts:.2f}%"

In [198]:
from typing import List


deal_id = 413629


@dataclass
class ProductDiscountStatement:
    """
    Discount statement for product or component
    """
    product_id: str
    hasSpecifiedDesiredPrice: bool
    desired_discount_per: Optional[float] = None
    desired_discount: [float] = None


@dataclass
class DealDiscountStatement:
    products_discount_statements: List[ProductDiscountStatement]


discount_statement = DiscountStatement(
    hasSpecifiedDesiredPrice=True
)

products_bought_before = get_products_bought_before(deal_id)

# has the client bought product before?
if len(products_bought_before) > 0:
    if check_margin_with_previous_prices(deal_id):
        print('Setup previous price')
    else:
        print('Setup 12% margin')
elif discount_statement.hasSpecifiedDesiredPrice:
    deal = get_latest_offer(deal_id)
    deal.desired_discount = 10.0  # Example desired discount for the entire deal
    result = apply_discounts_and_check_margin(deal)
    
    if isinstance(result, str):
        print(result)
    else:
        print(f"Updated Deal: {result}")
    
    

latest_offer = get_latest_offer(413629)


common_margin = calculate_common_margin(latest_offer)
common_margin

check_margin_with_previous_prices(413629)

38.14155391161025
Setup previous price
38.14155391161025


True

In [296]:
products_bought_before

[]

In [ ]:
451540_41673.0

In [219]:
deal_id = 451540
deal_info = client.offers_by_deal_id(deal_id)

print(client.prettify_history_response(deal_info))

ID: 343167
Request ID: 451540
Created: 2023-11-06T06:35:44+00:00
Last Sent: 2023-11-15T06:53:28+00:00
Processing Time: 1208684
Decision: 2
Link: index.php?option=com_oscrm&view=requisition&request_id=451540&Itemid=219
Target Price: 0
Target Price Solved: False
State: True
Proform: 55456
--------------------------



In [211]:
print(client.prettify_product_details(client.offer_products(deal_id)))

ID: 1168514
Articul: S260
Price: 155
Price Buy DE: 0
Description: TEMP ASSEMBLY
Rodax code: S260 PVA AG S HV D6 W3 A1 E405 N110 Z310 SC173
S260: Assembly with low screwed head in aluminum
PVA: Hi-tech spring loaded thermoplast
AG: ATEX Gas Ex ic ec
S: Single element
HV: High vibration resistance
D6: Sensor diameter 6 mm
W3: 3 wire electrical connection
A1: Accuracy class A
E405: Connection to thermowell ½" NPT
N110: Extension length 110 mm
Z310: Bore depth in thermowell 310 mm
SC173: Single conduit 1x M20x1.5
ATEX II 3G Ex ic/ec IIC T6...T1 Gc
According to drawing 2113.2117.450

+Q02031
CERTIFICATES
Q02031 EN 10204-2.1 Certificate of compliance to order
--------------------------



In [209]:
client?

Type:        APIClient
String form: <__main__.APIClient object at 0x10ac57e00>
Docstring:   <no docstring>

In [195]:
check_margin_with_previous_prices(413629)

38.14155391161025


True

In [199]:
products_bought_before = get_products_bought_before(451540)
products_bought_before

[]

In [159]:
# filtered_calculations = filter_calculations_by_notice_time(calcilation_response.content, notice_resp.content)
get_products_with_changed_price(calcilation_response.content, notice_resp.content)

{'1095279': <PriceState.PRICE_DECREASED: 'PriceDecreased'>}

In [63]:
deal_id = 454448
rsp = client.list_offers_by_deal_id(deal_id)


In [62]:
rsp = client.list_offers_by_client_id(98640)
# customer_purchase_history = rsp.content

rsp.content[0]

HistoryContent(id=382282, request=Request(id=509466, firm=Firm(id=98640, title='REFACCIONES INDUSTRIALES GABSA, SA DE CV', type='3', conversion='33.33', link='index.php?option=com_oscrm&view=firmform&id=98640&Itemid=219'), initials='EG', brand=Brand(id=12540, title='SMITEC', conversion='21.74', link='index.php?option=com_oscrm&view=brandform&id=12540&Itemid=219'), contractor=Contractor(id=5819, title='SMIPACK S.p.A.', link='index.php?option=com_oscrm&view=contractorform&id=5819&Itemid=216', created='2016-10-04T06:50:31+00:00', modified='2024-04-09T14:31:53+00:00'), curator=Curator(id=851, name='Adrian Dumitrascu', username='Adrian Dumitrascu', email='adu@famaga.de'), requestManagers=[[]], cost='626.15'), created='2024-05-27T15:24:45+00:00', lastsend='-0001-11-30T00:00:00+00:00', processing_time='', decision=0, link='index.php?option=com_oscrm&view=requisition&request_id=509466&Itemid=219', target_price=0, target_price_solved=False, state=True, proform=Proform(fam=None))

In [77]:
deal_id = 454448
rsp = client.offer_products(deal_id)

rsp

[ProductDetail(id=1174132, ri_id=855528, cc_id=539935, request_id=454448, brand_id=7101, articul='PCIPFM_025_1_1_DE', articul_id=822466, contractor_id=3620, count=10, price='196', price_buy_de=0, price_end='147.5', price_deliv=0, discount='50', discount_client=0, margin_ru=20, tax=7, price_buy_ru=196, price_buy=0, price_sell_ru=147.5, price_sell=0, unit='pcs', weight='0.4', planned_arrival_germany_from=0, planned_arrival_germany_to=0, confirmed_arrival_from=0, confirmed_arrival_to=0, checked_out=0, checked_out_time='0000-00-00 00:00:00', created_by=0, state=1, ordering=1019923, model_number='', description='', description_de='', brand_title='Procom Valves'),
 ProductDetail(id=1185369, ri_id=0, cc_id=539935, request_id=454448, brand_id=0, articul='', articul_id=0, contractor_id=3620, count=1, price='15', price_buy_de=0, price_end='22', price_deliv=0, discount='0', discount_client=0, margin_ru=20, tax=0, price_buy_ru=15, price_buy=0, price_sell_ru=22, price_sell=0, unit='pcs', weight='0'

In [ ]:
customer_purchase_history

In [484]:
import pandas as pd

# Extended knowledge map data with measurable parameters and human-readable instructions
data = {
    "Scenario": [
        "Customer previously purchased THIS product",
        "Customer has NOT previously purchased THIS product but specified a desired price",
        "Customer has NOT purchased THIS product and has NOT specified a desired price, BUT has purchased other products",
        "Customer has NEVER purchased before and has NOT specified a desired price",
        "Outcome - FULL discount refusal",
        "Outcome - Offering a 2% discount",
        "Instructions"  # This row will contain human-readable instructions
    ],
    "Condition": [
        "Profitability allows",
        "Margin allows",
        "Margin unchanged in previous deals",
        "First quotation for a large company",
        "Customer still buys",
        "Customer buys",
        "Describe the main condition to check for this scenario"
    ],
    "Action": [
        "Offer the same price as last time",
        "Offer a discount",
        "Refuse the discount",
        "Offer a 2% discount and ask for desired price",
        "Continue to refuse discounts",
        "Continue offering 2% where possible",
        "Specify the action to take if the main condition is met"
    ],
    "Alternative Condition": [
        "Profitability does not allow",
        "Margin does not allow",
        "Margin changed in previous deals - price in final offer is higher",
        "First quotation for a non-large company",
        "Customer does not buy",
        "Customer does not buy",
        "Describe the alternative condition to check if the main condition is not met"
    ],
    "Alternative Action": [
        "Set a 12% margin and explain supplier's price change",
        "Set a 10% margin and explain maximum discount",
        "Ignore the case",
        "Refuse the discount",
        "Offer a 2% discount next time and ask for desired price",
        "Offer a 3% discount next time if margin allows",
        "Specify the action to take if the alternative condition is met"
    ],
    "Additional Condition": [
        None,
        None,
        "Margin changed in previous deals - discount given",
        "More than 10 quotations without sales",
        None,
        None,
        "Describe any additional conditions that might apply"
    ],
    "Additional Action": [
        None,
        None,
        "Offer a 2% discount",
        "Set a 10% margin, candidate for blacklist",
        None,
        None,
        "Specify the action to take if the additional condition is met"
    ],
    "Further Condition": [
        None,
        None,
        None,
        "Several quotations with close request dates",
        None,
        None,
        "Describe any further conditions that might apply"
    ],
    "Further Action": [
        None,
        None,
        None,
        "Propose discount for multiple quotations, ask for desired price, hand over to manager",
        None,
        None,
        "Specify the action to take if the further condition is met"
    ],
    "desired_price_specified": [
        False,
        True,
        False,
        False,
        None,
        None,
        "Is the desired price specified by the customer? (True/False)"
    ],
    "previously_purchased_this_product": [
        True,
        False,
        False,
        False,
        None,
        None,
        "Has the customer previously purchased this product? (True/False)"
    ],
    "previously_purchased_other_products": [
        False,
        False,
        True,
        False,
        None,
        None,
        "Has the customer previously purchased other products? (True/False)"
    ],
    "profitability_allows": [
        True,
        None,
        None,
        None,
        None,
        None,
        "Does profitability allow offering the same price? (True/False)"
    ],
    "margin_allows": [
        None,
        True,
        None,
        None,
        None,
        None,
        "Does the margin allow offering a discount? (True/False)"
    ],
    "margin_unchanged_in_previous_deals": [
        None,
        None,
        True,
        None,
        None,
        None,
        "Was the margin unchanged in previous deals? (True/False)"
    ],
    "price_in_final_offer_higher": [
        None,
        None,
        False,
        None,
        None,
        None,
        "Was the price in the final offer higher than the initial offer? (True/False)"
    ],
    "discount_given_previously": [
        None,
        None,
        True,
        None,
        None,
        None,
        "Was a discount given previously? (True/False)"
    ],
    "first_quotation_large_company": [
        None,
        None,
        None,
        True,
        None,
        None,
        "Is this the first quotation for a large company? (True/False)"
    ],
    "first_quotation_non_large_company": [
        None,
        None,
        None,
        False,
        None,
        None,
        "Is this the first quotation for a non-large company? (True/False)"
    ],
    "more_than_10_quotations_without_sales": [
        None,
        None,
        None,
        True,
        None,
        None,
        "Has the customer received more than 10 quotations without any sales? (True/False)"
    ],
    "several_quotations_close_dates": [
        None,
        None,
        None,
        True,
        None,
        None,
        "Are there several quotations with close request dates? (True/False)"
    ]
}

df = pd.DataFrame(data)

df

,Scenario,Condition,Action,Alternative Condition,Alternative Action,Additional Condition,Additional Action,Further Condition,Further Action,desired_price_specified,...,previously_purchased_other_products,profitability_allows,margin_allows,margin_unchanged_in_previous_deals,price_in_final_offer_higher,discount_given_previously,first_quotation_large_company,first_quotation_non_large_company,more_than_10_quotations_without_sales,several_quotations_close_dates
0,Customer previously purchased THIS product,Profitability allows,Offer the same price as last time,Profitability does not allow,Set a 12% margin and explain supplier's price ...,None,None,None,None,False,...,False,True,None,None,None,None,None,None,None,None
1,Customer has NOT previously purchased THIS pro...,Margin allows,Offer a discount,Margin does not allow,Set a 10% margin and explain maximum discount,None,None,None,None,True,...,False,None,True,None,None,None,None,None,None,None
2,Customer has NOT purchased THIS product and ha...,Margin unchanged in previous deals,Refuse the discount,Margin changed in previous deals - price in fi...,Ignore the case,Margin changed in previous deals - discount given,Offer a 2% discount,None,None,False,...,True,None,None,True,False,True,None,None,None,None
3,Customer has NEVER purchased before and has NO...,First quotation for a large company,Offer a 2% discount and ask for desired price,First quotation for a non-large company,Refuse the discount,More than 10 quotations without sales,"Set a 10% margin, candidate for blacklist",Several quotations with close request dates,"Propose discount for multiple quotations, ask ...",False,...,False,None,None,None,None,None,True,False,True,True
4,Outcome - FULL discount refusal,Customer still buys,Continue to refuse discounts,Customer does not buy,Offer a 2% discount next time and ask for desi...,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,Outcome - Offering a 2% discount,Customer buys,Continue offering 2% where possible,Customer does not buy,Offer a 3% discount next time if margin allows,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,Instructions,Describe the main condition to check for this ...,Specify the action to take if the main conditi...,Describe the alternative condition to check if...,Specify the action to take if the alternative ...,Describe any additional conditions that might ...,Specify the action to take if the additional c...,Describe any further conditions that might apply,Specify the action to take if the further cond...,Is the desired price specified by the customer...,...,Has the customer previously purchased other pr...,Does profitability allow offering the same pri...,Does the margin allow offering a discount? (Tr...,Was the margin unchanged in previous deals? (T...,Was the price in the final offer higher than t...,Was a discount given previously? (True/False),Is this the first quotation for a large compan...,Is this the first quotation for a non-large co...,Has the customer received more than 10 quotati...,Are there several quotations with close reques...


# Flattened Dataframes

In [579]:
import requests
import logging
import pandas as pd
from typing import Dict, Any

BASE_URL = "https://api.famaga.org"

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


class APIClientV2:
    def __init__(self, token: str) -> None:
        self.token = token

    def _get_headers(self) -> Dict[str, str]:
        return {"Authorization": f"Bearer {self.token}"}

    def _handle_response(self, response: requests.Response) -> Dict:
        try:
            response.raise_for_status()
            print(f"Request to {response.url} succeeded with status code {response.status_code}")
            return response.json()
        except requests.exceptions.HTTPError as e:
            print(f"HTTP error occurred: {e}")
            raise
        except requests.exceptions.RequestException as e:
            print(f"Error occurred: {e}")
            raise

    @stored
    def _get(self, endpoint: str, params: Dict = None) -> Dict:
        url = f"{BASE_URL}{endpoint}"
        response = requests.get(url, headers=self._get_headers(), params=params)
        return self._handle_response(response)

    def _flatten_json(self, y: Dict[str, Any], parent_key: str = '', sep: str = '_') -> Dict[str, Any]:
        items = []
        for k, v in y.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                items.extend(self._flatten_json(v, new_key, sep=sep).items())
            elif isinstance(v, list):
                for i, item in enumerate(v):
                    items.extend(self._flatten_json({f"{new_key}_{i}": item}).items())
            else:
                items.append((new_key, v))
        return dict(items)

    def offers_by_client_id(self, client_id: int) -> pd.DataFrame:
        data = self._get('/requisition', params={'client': client_id})
        flattened_data = [self._flatten_json(content) for content in data["content"]]
        return pd.DataFrame(flattened_data)

    def offers_by_deal_id(self, deal_id: int) -> pd.DataFrame:
        data = self._get('/requisition', params={'request': deal_id})
        flattened_data = [self._flatten_json(content) for content in data["content"]]
        return pd.DataFrame(flattened_data)

    def offer_products(self, offer_id: int) -> pd.DataFrame:
        data = self._get(f'/requisition/{offer_id}/products')
        flattened_data = [self._flatten_json(product) for product in data]
        return pd.DataFrame(flattened_data)

    def get_notice_mail(self, request_id: int) -> pd.DataFrame:
        data = self._get('/requisition-client-history', params={'request': request_id})
        flattened_data = [self._flatten_json(content) for content in data["content"]]
        return pd.DataFrame(flattened_data)

    def get_calculation_history(self, request_id: int) -> pd.DataFrame:
        data = self._get('/calculation-history', params={'request': request_id})
        normalized_data = []

        product_data = []
    
        for entry in data["content"]:
            if entry["version_data"]:
                calculation_id = entry["id"]
                request_id = entry["request"]["id"]
                created_at = entry["created_at"]
    
                for product in entry["version_data"]["products"]:
                    product_entry = {
                        "request_id": request_id,
                        "calculation_id": calculation_id,
                        "created_at": created_at,
                        **product
                    }
                    product_data.append(product_entry)
        
        # Create DataFrame
        product_df = pd.DataFrame(product_data)
        return product_df


    def to_dict(self):
        return {
            "token": self.token
        }

In [580]:
client = APIClientV2('YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw')

In [503]:
client_id

41692

In [519]:
client_purchase_history_df = client.offers_by_client_id(client_id)

client_purchase_history_df.columns.tolist()

['id',
 'request_id',
 'request_firm_id',
 'request_firm_title',
 'request_firm_type',
 'request_firm_conversion',
 'request_firm_link',
 'request_initials',
 'request_brand_id',
 'request_brand_title',
 'request_brand_conversion',
 'request_brand_link',
 'request_contractor_id',
 'request_contractor_title',
 'request_contractor_link',
 'request_contractor_created',
 'request_contractor_modified',
 'request_curator_id',
 'request_curator_name',
 'request_curator_username',
 'request_curator_email',
 'request_cost',
 'created',
 'lastsend',
 'processing_time',
 'decision',
 'link',
 'target_price',
 'target_price_solved',
 'state',
 'proform_fam']

In [494]:
client_purchase_history_df.columns.tolist()

['id',
 'request_id',
 'request_firm_id',
 'request_firm_title',
 'request_firm_type',
 'request_firm_conversion',
 'request_firm_link',
 'request_initials',
 'request_brand_id',
 'request_brand_title',
 'request_brand_conversion',
 'request_brand_link',
 'request_contractor_id',
 'request_contractor_title',
 'request_contractor_link',
 'request_contractor_created',
 'request_contractor_modified',
 'request_curator_id',
 'request_curator_name',
 'request_curator_username',
 'request_curator_email',
 'request_cost',
 'created',
 'lastsend',
 'processing_time',
 'decision',
 'link',
 'target_price',
 'target_price_solved',
 'state',
 'proform_fam']

In [496]:
customer_offer_products_df = client.offer_products(450789)

In [581]:
import pandas as pd

# Step 1: Get unique request_id from client_purchase_history_df
client_purchase_history_df = client.offers_by_client_id(client_id)
unique_request_ids = client_purchase_history_df['request_id'].unique()

# Step 2: Build common DataFrame from request_ids with offer_products
offer_products_list = []
for request_id in unique_request_ids:
    offer_products_df = client.offer_products(request_id)
    offer_products_df['request_id'] = request_id  # Add request_id to each row
    offer_products_list.append(offer_products_df)
common_offer_products_df = pd.concat(offer_products_list, ignore_index=True)

# Step 3: Build common DataFrame from request_ids with get_notice_mail
notice_mail_list = []
for request_id in unique_request_ids:
    notice_mail_df = client.get_notice_mail(int(request_id))
    notice_mail_df['request_id'] = request_id  # Add request_id to each row
    notice_mail_list.append(notice_mail_df)
common_notice_mail_df = pd.concat(notice_mail_list, ignore_index=True)

# Step 4: Build common DataFrame from request_ids with get_calculation_history
calculation_history_list = []
for request_id in unique_request_ids:
    print(request_id)
    calculation_history_df = client.get_calculation_history(int(request_id))
    calculation_history_df['request_id'] = request_id  # Add request_id to each row
    calculation_history_list.append(calculation_history_df)
common_calculation_history_df = pd.concat(calculation_history_list, ignore_index=True)

450789
455121
453534
453931
448583
425906
421312
419563
419954
412938
416367
411849
411847
411846
411848
407758
400067
400812
399215
394252


In [578]:
client.get_calculation_history(int(450789))

In [551]:
common_notice_mail_df.columns.tolist()

['id', 'request_id', 'type', 'user_id', 'created_at', 'note', 'status']

In [637]:
client_purchase_history_df[~client_purchase_history_df['decision'].isin([1, 2])]

,id,request_id,request_firm_id,request_firm_title,request_firm_type,request_firm_conversion,request_firm_link,request_initials,request_brand_id,request_brand_title,...,request_cost,created,lastsend,processing_time,decision,link,target_price,target_price_solved,state,proform_fam
0,346245,450789,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,EG,1069,Johnson Pump (brand of SPX FLOW),...,107.22,2023-11-21T13:36:28+00:00,2023-11-21T14:22:22+00:00,,8,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN
1,344754,455121,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,AKA,13437,Bosch Rexroth Oil Control,...,1796.25,2023-11-14T08:59:01+00:00,2023-11-14T15:22:01+00:00,,5,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN
2,344604,453534,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,EG,1612,orange research,...,3027.5,2023-11-13T13:36:20+00:00,2023-11-14T13:27:19+00:00,,5,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN
3,344239,453931,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,AKA,13435,Bosch Rexroth Industriehydraulik,...,3081.25,2023-11-10T09:10:10+00:00,2023-11-12T09:12:03+00:00,,5,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN
5,325101,425906,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,KP,988,BEKO TECHNOLOGIES / BEKOMAT,...,806.88,2023-07-27T08:10:27+00:00,2023-07-27T14:02:35+00:00,,5,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN
7,321127,419563,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,KP,13330,Inbal Valves,...,24811.91,2023-07-07T06:52:31+00:00,2023-07-12T13:59:23+00:00,,5,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN
8,320800,419954,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,KP,1805,Schischek,...,2857.5,2023-07-05T12:04:55+00:00,2023-07-05T12:32:29+00:00,,5,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN
9,319319,412938,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,KP,12997,Fawcett Christie Hydraulics,...,76732.75,2023-06-27T14:57:36+00:00,2023-06-28T14:37:29+00:00,,5,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN
10,318580,416367,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,KP,4459,CEWE INSTRUMENT,...,10300.99,2023-06-23T12:14:30+00:00,2023-07-05T15:49:10+00:00,,5,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN
11,318391,411849,41692,Storm Procurement Ltd,3,5.56,index.php?option=com_oscrm&view=firmform&id=41...,KP,20546,Teledyne Gas and Flame Detection,...,15867.8,2023-06-22T14:03:16+00:00,2023-06-22T14:33:27+00:00,,5,index.php?option=com_oscrm&view=requisition&re...,0,False,True,NaN


In [638]:
common_notice_mail_df

,id,request_id,type,user_id,created_at,note,status
0,733328,450789,notice,783,2023-12-06 22:51:48+00:00,Kanesh.Dattani@storm-procurement.com,1
1,730989,450789,notice,783,2023-11-29 21:04:55+00:00,Kanesh.Dattani@storm-procurement.com,1
2,727566,450789,mail,783,2023-11-21 14:22:22+00:00,Angebot_450789.pdf,2
3,733331,455121,notice,783,2023-12-06 23:03:37+00:00,Davis.Gurung@storm-procurement.com,1
4,730995,455121,notice,783,2023-11-29 21:20:17+00:00,Davis.Gurung@storm-procurement.com,1
...,...,...,...,...,...,...,...
95,656205,394252,notice,783,2023-04-24 14:43:59+00:00,evgeny.kheyman@storm-procurement.com,2
96,655068,394252,notice,783,2023-04-19 16:54:29+00:00,evgeny.kheyman@storm-procurement.com,2
97,651549,394252,notice,814,2023-04-10 11:58:30+00:00,evgeny.kheyman@storm-procurement.com,2
98,649613,394252,notice,814,2023-04-03 13:36:12+00:00,evgeny.kheyman@storm-procurement.com,2


In [582]:
common_calculation_history_df.columns.tolist()

['request_id',
 'calculation_id',
 'created_at',
 'id',
 'tax',
 'unit',
 'count',
 'weight',
 'articul',
 'expense',
 'brand_id',
 'discount',
 'margin_ru',
 'description',
 'price_deliv',
 'model_number',
 'price_buy_de',
 'price_buy_ru',
 'price_sell_ru',
 'description_de',
 'cc_id',
 'price',
 'ri_id',
 'state',
 'ordering',
 'price_buy',
 'price_end',
 'articul_id',
 'created_by',
 'price_sell',
 'brand_title',
 'checked_out',
 'contractor_id',
 'discount_client',
 'checked_out_time',
 'confirmed_arrival_to',
 'confirmed_arrival_from',
 'planned_arrival_germany_to',
 'planned_arrival_germany_from']

In [592]:
common_notice_mail_df['created_at'] = pd.to_datetime(common_notice_mail_df['created_at'])
common_calculation_history_df['created_at'] = pd.to_datetime(common_calculation_history_df['created_at'])

# Function to filter rows within the specified timedelta
def is_within_timedelta(row, notice_times, delta):
    calc_time = row['created_at']
    return any((calc_time >= (notice_time - delta)) and (calc_time <= (notice_time + delta)) for notice_time in notice_times)

# Collect all notice times in a dictionary by request_id
notice_times_dict = common_notice_mail_df.groupby('request_id')['created_at'].apply(list).to_dict()

# Apply the filter
timedelta_value = timedelta(minutes=5)
filtered_rows = []

for _, row in common_calculation_history_df.iterrows():
    notice_times = notice_times_dict.get(row['request_id'], [])
    if is_within_timedelta(row, notice_times, timedelta_value):
        filtered_rows.append(row)

filtered_df = pd.DataFrame(filtered_rows)

In [627]:
# Function to find the first and latest records
def get_first_and_latest(df):
    first_record = df.loc[df['created_at'].idxmin()]
    latest_record = df.loc[df['created_at'].idxmax()]
    return first_record, latest_record

# Group by request_id and find changes in price
price_changed_rows = []

for request_id, group in filtered_df.groupby('request_id'):
    first_record, latest_record = get_first_and_latest(group)
    if first_record['count'] == latest_record['count'] and first_record['price_sell_ru'] != latest_record['price_sell_ru']:
        price_changed_rows.append(latest_record)

price_changed_df = pd.DataFrame(price_changed_rows)

price_changed_df.iloc[1:].head()

,request_id,calculation_id,created_at,id,tax,unit,count,weight,articul,expense,...,price_sell,brand_title,checked_out,contractor_id,discount_client,checked_out_time,confirmed_arrival_to,confirmed_arrival_from,planned_arrival_germany_to,planned_arrival_germany_from
302,400812,2428853,2023-04-18 14:44:03+00:00,1055160,7,pcs,4,2.5,DPT521- 401,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,407758,2480435,2023-05-23 15:07:48+00:00,1075950,7,pcs,2,2.363,000782,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,411847,2508176,2023-06-08 14:49:41+00:00,1086295,7,pcs,30,0,50107579,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,412938,2543960,2023-06-28 14:36:10+00:00,1096131,7,pcs,4,18,21393-C90,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,421312,2639206,2023-08-28 12:50:52+00:00,1104384,7,pcs,2,0.75,0820215114,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [629]:
filtered_df[(filtered_df['request_id'] == 407758)][['price_sell_ru', 'count', 'created_at', 'articul', 'discount', 'margin_ru']]

,price_sell_ru,count,created_at,articul,discount,margin_ru
250,1135.37,2,2023-05-23 15:07:48+00:00,000782,0,20
251,306.04,2,2023-05-23 15:07:48+00:00,000169,0,20
252,1135.37,2,2023-05-23 15:07:42+00:00,000782,0,20
253,306.04,2,2023-05-23 15:07:42+00:00,000169,0,20
254,1123.56,2,2023-05-23 15:06:22+00:00,000782,0,20
255,302.86,2,2023-05-23 15:06:22+00:00,000169,0,20


In [606]:
filtered_df[['request_id', 'calculation_id', 'count', 'articul', 'price_end']]

,request_id,calculation_id,count,articul,price_end
6,455121,2747455,3,R930067629,NaN
13,453931,2743649,1,R902137843,NaN
17,448583,2717648,4,HPQ330328S,NaN
20,425906,2595161,1,2000020,NaN
36,421312,2639206,2,0820215114,NaN
...,...,...,...,...,...
310,399215,2428116,2,QD2500,NaN
311,399215,2428115,2,ND2500,NaN
312,399215,2428115,2,QD2500,NaN
333,394252,2394598,1,KL120,NaN


In [590]:
common_calculation_history_df

,request_id,calculation_id,created_at,id,tax,unit,count,weight,articul,expense,...,price_sell,brand_title,checked_out,contractor_id,discount_client,checked_out_time,confirmed_arrival_to,confirmed_arrival_from,planned_arrival_germany_to,planned_arrival_germany_from
0,450789,2757256,2023-11-21 14:13:26+00:00,1178108,7,pcs,1,1,15957,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,450789,2757245,2023-11-21 14:08:40+00:00,1178108,7,pcs,1,1,15957,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,450789,2757107,2023-11-21 13:36:28+00:00,1178108,7,pcs,1,1,15957,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,450789,2757104,2023-11-21 13:36:08+00:00,1178108,7,pcs,1,1,15957,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,450789,2757102,2023-11-21 13:35:52+00:00,1178108,7,pcs,1,1,15957,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336,394252,2394509,2023-03-24 13:02:58+00:00,1041991,7,pcs,1,1,KL120,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
337,394252,2394508,2023-03-24 13:02:32+00:00,1041991,7,pcs,1,1,KL120,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,394252,2394505,2023-03-24 13:02:10+00:00,1041991,7.00,pcs,1,0.000,9317903,NaN,...,0.00,Doedijns,0,32509,0.00,0000-00-00 00:00:00,0,0,0,0
339,394252,2394504,2023-03-24 13:02:08+00:00,1041991,7.00,pcs,1,0.000,9317903,NaN,...,0.00,Doedijns,0,32509,0.00,0000-00-00 00:00:00,0,0,0,0


In [ ]:
import pandas as pd
import json
from tabulate import tabulate

markdown_table = tabulate(common_calculation_history_df.head(2), headers='keys', tablefmt='pipe')
print(markdown_table)

In [554]:
common_offer_products_df.columns.tolist()

['id',
 'ri_id',
 'cc_id',
 'request_id',
 'brand_id',
 'articul',
 'articul_id',
 'model_number',
 'description',
 'description_de',
 'contractor_id',
 'count',
 'price',
 'price_buy_de',
 'price_end',
 'price_deliv',
 'discount',
 'discount_client',
 'margin_ru',
 'tax',
 'price_buy_ru',
 'price_buy',
 'price_sell_ru',
 'price_sell',
 'unit',
 'weight',
 'planned_arrival_germany_from',
 'planned_arrival_germany_to',
 'confirmed_arrival_from',
 'confirmed_arrival_to',
 'checked_out',
 'checked_out_time',
 'created_by',
 'state',
 'ordering',
 'brand_title']